[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation)  
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [53]:
## 1. 모듈 import
from operator import itemgetter
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import trim_messages
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI


## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
store = {}

## 4. 함수 정의 : 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의 
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다. 
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''), 
    ('placeholder','{chat_history}'),
    ('user', '{query}'),
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 6. ChatOpenAI 인스턴스 생성 : 모델 생성(지정)
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 7. trim 설정
trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

## 8. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history')| trimmer)
    | prompt 
    | llm) 

## 9. chain 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)
## 10. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>>')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {"session_id": 'abc'}}
    )
    print('\n' + '=' * 50 + '\n')

이력서를 작성하기 위해서는 먼저 기본적인 개인 정보와 연락처를 포함해야 합니다. 그런 다음, 경력, 학력, 기술 및 역량, 수상 경력 등 중요한 정보를 체계적으로 정리합니다. 각 섹션이 명확하게 구분되도록 하여 고용주가 쉽게 확인할 수 있도록 해야 합니다. 경험과 성과를 구체적으로 설명하며, 가능한 경우 수치로 나타내 영향력을 강조하는 것도 좋습니다. 이제 제공하신 정보를 바탕으로 이력서를 작성하겠습니다. 필요한 정보를 제공해 주세요.

물론입니다. 이력서를 작성하기 위해 필요한 몇 가지 정보가 필요합니다. 아래 항목들에 대해 알려주시면 감사하겠습니다:

1. 이름
2. 연락처 정보 (전화번호, 이메일)
3. 주소 (선택 사항)
4. 직무 목표 (희망 직무 또는 목표)
5. 학력 (학교 이름, 전공, 졸업 연도)
6. 경력 (회사 이름, 직무, 근무 기간, 주요 업무 및 성과)
7. 기술 및 자격증
8. 수상 및 인증
9. 기타 이력서에 포함하고 싶은 내용 (예: 특정 프로젝트, 봉사 활동 등)

이 정보를 바탕으로 이력서를 작성하도록 하겠습니다.

박보검

[연락처]
이메일: example@email.com
전화번호: 010-1234-5678
주소: 서울특별시 강남구

[프로필]
신입 웹 개발자로서 최신 기술과 트렌드를 적극적으로 학습하며, 높은 수준의 문제 해결 능력과 창의적인 사고를 겸비한 지원자입니다. Python과 웹 디자인에 대한 깊은 이해를 바탕으로 사용자 친화적이고 효율적인 웹 솔루션을 개발하는 데 중점을 두고 있습니다.

[기술 스택]
- 프로그래밍 언어: Python 
- 웹 기술: HTML, CSS
- 기타: LLM (Large Language Models)

[학력]
- [대학교 이름], 컴퓨터공학과 졸업 (졸업년도: 2023)

[프로젝트 경험]
1. **웹사이트 디자인 프로젝트**
   - 기술: HTML5, CSS3
   - 설명: 사용자 친화적인 인터페이스를 설계하고 구현하여 UX를 개선했습니다. 반응형 웹 디자인을 통해 다양한 기

In [54]:
get_history('abc')

InMemoryChatMessageHistory(messages=[HumanMessage(content='이력서를 어떻게 작성하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='이력서를 작성하기 위해서는 먼저 기본적인 개인 정보와 연락처를 포함해야 합니다. 그런 다음, 경력, 학력, 기술 및 역량, 수상 경력 등 중요한 정보를 체계적으로 정리합니다. 각 섹션이 명확하게 구분되도록 하여 고용주가 쉽게 확인할 수 있도록 해야 합니다. 경험과 성과를 구체적으로 설명하며, 가능한 경우 수치로 나타내 영향력을 강조하는 것도 좋습니다. 이제 제공하신 정보를 바탕으로 이력서를 작성하겠습니다. 필요한 정보를 제공해 주세요.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8'}, id='run-ad395dcd-3d61-4c05-9576-59d1fd149e05-0'), HumanMessage(content='이력서를 작성해주세요.', additional_kwargs={}, response_metadata={}), AIMessage(content='물론입니다. 이력서를 작성하기 위해 필요한 몇 가지 정보가 필요합니다. 아래 항목들에 대해 알려주시면 감사하겠습니다:\n\n1. 이름\n2. 연락처 정보 (전화번호, 이메일)\n3. 주소 (선택 사항)\n4. 직무 목표 (희망 직무 또는 목표)\n5. 학력 (학교 이름, 전공, 졸업 연도)\n6. 경력 (회사 이름, 직무, 근무 기간, 주요 업무 및 성과)\n7. 기술 및 자격증\n8. 수상 및 인증\n9. 기타 이력서에 포함하고 싶은 내용 (예: 특정 프로젝트, 봉사 활동 등)\n\n이 정보를 바탕으로 이력서를 작성하도록 하겠습니다.', additional_kwa

In [55]:
for message in get_history('abc').messages:
    print(f'[{message.type.upper()}]:{message.content}\n')

[HUMAN]:이력서를 어떻게 작성하나요?

[AI]:이력서를 작성하기 위해서는 먼저 기본적인 개인 정보와 연락처를 포함해야 합니다. 그런 다음, 경력, 학력, 기술 및 역량, 수상 경력 등 중요한 정보를 체계적으로 정리합니다. 각 섹션이 명확하게 구분되도록 하여 고용주가 쉽게 확인할 수 있도록 해야 합니다. 경험과 성과를 구체적으로 설명하며, 가능한 경우 수치로 나타내 영향력을 강조하는 것도 좋습니다. 이제 제공하신 정보를 바탕으로 이력서를 작성하겠습니다. 필요한 정보를 제공해 주세요.

[HUMAN]:이력서를 작성해주세요.

[AI]:물론입니다. 이력서를 작성하기 위해 필요한 몇 가지 정보가 필요합니다. 아래 항목들에 대해 알려주시면 감사하겠습니다:

1. 이름
2. 연락처 정보 (전화번호, 이메일)
3. 주소 (선택 사항)
4. 직무 목표 (희망 직무 또는 목표)
5. 학력 (학교 이름, 전공, 졸업 연도)
6. 경력 (회사 이름, 직무, 근무 기간, 주요 업무 및 성과)
7. 기술 및 자격증
8. 수상 및 인증
9. 기타 이력서에 포함하고 싶은 내용 (예: 특정 프로젝트, 봉사 활동 등)

이 정보를 바탕으로 이력서를 작성하도록 하겠습니다.

[HUMAN]:이름 : 박보검 신입 웹개발 기술: llm, python, css, html입니다.

[AI]:박보검

[연락처]
이메일: example@email.com
전화번호: 010-1234-5678
주소: 서울특별시 강남구

[프로필]
신입 웹 개발자로서 최신 기술과 트렌드를 적극적으로 학습하며, 높은 수준의 문제 해결 능력과 창의적인 사고를 겸비한 지원자입니다. Python과 웹 디자인에 대한 깊은 이해를 바탕으로 사용자 친화적이고 효율적인 웹 솔루션을 개발하는 데 중점을 두고 있습니다.

[기술 스택]
- 프로그래밍 언어: Python 
- 웹 기술: HTML, CSS
- 기타: LLM (Large Language Models)

[학력]
- [대학교 이름], 컴퓨터공학과 졸업 (졸업년도: 2023)

